# Assumption-based Truth Maintenance System

You are given an implementation of ATMS written in Python.  An example KB has been given for you to test its functionality.

1. Determine what each part of the code does, making modifications where necessary to determine how everything works.  In particular, be prepared to explain the following:

    1. What data is stored in each of the classes defined and why does this data need to be stored?
    2. What is the purpose of the itertools.product() function inside the while loop?
    3. When performing optimizations on the KB, what is the meaning of a > b when a and b are set objects?
    4. Why are frozenset objects used extensively throughout the code?
    5. We see the asterisk used inside functions calls in a couple of places (e.g. frozenset.union(*C[u])).  What is the purpose of the asterisk?


2. Enter in the plumbing KB from the previous activity, using the appropriate data types to literals and horn clauses

3. Consider that we know that the supply pressure to the house is normal (p_1 is given)  Add this unit clause to your KB.

4. We have the follwing assumables: {on_t1, on_t2, on_t3, off_t1, off_t2, off_t3, plugged_sink, plugged_bath, unplugged_sink, unplugged_bath, blocked_d1, blocked_d2, blocked_d3, clear_d1, clear_d2, clear_d3, leaky_t1, leaky_t2, leaky_t3}.  Use the code find all of the minimal explanations for a wet bathroom floor in terms of these assumables.  How many of these correspond to a fault somewhere in the plumbing system?

### Fault-Diagnosis Implementation

In [1]:
# ATMS Python Implementation
# code by: R. Shekhar
# CS152 - Lesson 8.1

import itertools

################ Data Type Definitions ###################

# Define a datatype to represent a logical literal
class Literal:
    def __init__(self,name='',description=''):
        self.name = name
        self.description = description

    def __repr__(self):
        return self.__str__()

    def __str__(self):
        return self.name

# Define a datatype to represent a horn clause
class HornClause:
    def __init__(self,conclusion,premise):
        self.premise = frozenset(premise) # body
        self.conclusion = conclusion # head

    def __eq__(self,other): # equivalance
        return (self.premise == other.premise) and (self.conclusion == other.conclusion)

    def __hash__(self): # hashing to a store table
        return hash((self.premise, self.conclusion))

    def __repr__(self):
         return self.__str__()

    def __str__(self):
        if not self.premise:
            return self.conclusion.__str__()
        else:
            return self.conclusion.__str__() + u" \u21D0 " + u" \u2227 ".join("%s" % s.__str__() for s in self.premise)

In [4]:
################ Test Knowledge Base ###################
# This is just an example!
# You will need to write the rules for the plumbing system to replace these ones.

# Define the literals that will be used for inference
bronchitis = Literal('bronchitis')
influenza = Literal('influenza')
smokes = Literal('smokes')
coughing = Literal('coughing')
wheezing = Literal('wheezing')
fever = Literal('fever')
infection = Literal('infection')
sore_throat = Literal('sore_throat')
nonsmoker = Literal('nonsmoker')
dyspnea = Literal('dyspnea')
asthma = Literal('asthma')
false = Literal('false')



# Define the assumables
assumables_set = {influenza,infection,smokes,nonsmoker}

# Define the KB (atomic clauses can be represented with an empty premise)
KB = {HornClause(bronchitis,[influenza]),
      HornClause(bronchitis,[smokes]),
      HornClause(coughing,[bronchitis]),
      HornClause(wheezing,[bronchitis]),
      HornClause(fever,[influenza]),
      HornClause(fever,[infection]),
      HornClause(sore_throat,[influenza]),
      HornClause(false,[smokes,nonsmoker]),
      HornClause(dyspnea,[influenza,smokes]),
      HornClause(fever,[]), # Atomic clause
      }

In [3]:
################ ATMS Implementation ###################

# store the goal clauses in a separate set
goal_clauses = {c for c in KB if c.conclusion == false}
# Duplicate the KB for later manipulation
all_clauses = KB.copy()

# Define the assumables list that will be updated on each iteration of ATMS
C = {a:{frozenset([a])} for a in assumables_set}

# Iterate until we reach a fixed point (cannot infer anything more)
while True:
    # Find all the clauses we can infer (i.e. those whose premise are in the list of inferred literals C)
    inferred_clauses = {c for c in all_clauses if c.premise.issubset(C)}
    #Exit if we cannot infer anything more
    if not inferred_clauses: break
    # Loop over all clauses inferred those symbols inferred up to this point
    for clause in inferred_clauses:
            # Remove the clause from the list of all clauses
            all_clauses.remove(clause)
            # If this is the first time the conclusion has been drawn, initialize it.
            if clause.conclusion not in C: C[clause.conclusion] = set()
            # If the premise of the clause is empty, we don't need to find any combination of assumable sets
            if clause.premise:
                assumable_set = {frozenset(C[u]) for u in clause.premise}
                assumable_combos = {frozenset.union(*u) for u in itertools.product(*assumable_set)}
            else: assumable_combos = {frozenset()}

            # Optimizations
            # 1. Check if any of the assumables superset those already in the list itself or already recorded for this literal and remove them
            assumable_combos = {a for a in assumable_combos if not any (a > b for b in assumable_combos)}
            assumable_combos = {a for a in assumable_combos if not any (a > b for b in C[clause.conclusion])}
            # 2. Check if any subset has already been falsified by the goal clauses, and remove them
            assumable_combos = {a for a in assumable_combos if clause.conclusion is false or not any (a >= b.premise for b in goal_clauses)}
            # 3. Check if any of the assumables sets are a subset those already on the list and modify the list (a more minimal explanation has been found, so can replace the one already there)
            C[clause.conclusion].difference_update({a for a in C[clause.conclusion] if any (a > b for b in assumable_combos)})

            if assumable_combos: C[clause.conclusion].update(assumable_combos)

# Print out the minimal explanations for each clause
print("====ATMS Results====")
print("Assumables: " + str(assumables_set) + "\n")
print("KB: " + str(KB) + "\n")
given_literals = {c for c in KB if not c.premise}
if given_literals: print("Literals given directly from KB: " + str(given_literals) + "\n")
explanable_literals = [c for c in C if c != false and c not in assumables_set and C[c]]
max_len = max([len(l.name) for l in explanable_literals])
print("-------Minimal Explanations-------")

if false in C: max_len = max(max_len,len("conflicts"))
for c in explanable_literals:
    print("{1:<{0}}: {2}".format(max_len,c.name,", ".join(["{" + u" \u2227 ".join([str(x) for x in t]) + "}" for t in C[c]]) + "."))

if false in C: print("\n{1:<{0}}: {2}".format(max_len,"conflicts", ",".join(["{" + u" \u2227 ".join([str(x) for x in t]) + "}" for t in C[false]]) + "."))

====ATMS Results====
Assumables: {smokes, influenza, nonsmoker, infection}

KB: {fever ⇐ influenza, fever, wheezing ⇐ bronchitis, false ⇐ smokes ∧ nonsmoker, sore_throat ⇐ influenza, fever ⇐ infection, coughing ⇐ bronchitis, bronchitis ⇐ influenza, dyspnea ⇐ smokes ∧ influenza, bronchitis ⇐ smokes}

Literals given directly from KB: {fever}

-------Minimal Explanations-------
fever      : {}.
sore_throat: {influenza}.
bronchitis : {influenza}, {smokes}.
dyspnea    : {smokes ∧ influenza}.
wheezing   : {influenza}, {smokes}.
coughing   : {influenza}, {smokes}.

conflicts  : {smokes ∧ nonsmoker}.
